### neural network: tagId and relevance to rating

#### import the necessary python libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

#### read genome-scores and create a pandas dataframe for train_X

In [2]:
file_x = pd.read_csv('dataset/genome-scores.csv')
stacked_df_x = pd.DataFrame(file_x)

#### read train_ratings and create a pandas dataframe for train_Y

In [3]:
file_y = pd.read_csv('dataset/train_ratings.csv')
stacked_df_y = pd.DataFrame(file_y)

#### filter both dataframes so that both contain the same movieID's (i.e. filter by the intersection of the movieID's they contain)

In [4]:
filtered_stacked_df_y = stacked_df_y[stacked_df_y.movieId.isin(stacked_df_x.movieId)]
filtered_stacked_df_x = stacked_df_x[stacked_df_x.movieId.isin(stacked_df_y.movieId)]

#### convert train_X's dataframe into a tensor

In [5]:
matrix_df_x = filtered_stacked_df_x.pivot(index='movieId', columns='tagId', values='relevance')
np_train_X = matrix_df_x.values
train_X = torch.from_numpy(np_train_X).float()

#### convert train_Y's dataframe into a tensor

In [6]:
matrix_df_y = filtered_stacked_df_y.pivot(index='movieId', columns='userId', values='rating')
vector_df_y = matrix_df_y.mode(axis='columns')
np_train_Y = vector_df_y.iloc[:,0].values
train_Y = torch.from_numpy(np_train_Y).float()

In [9]:
print(train_X.shape)
print(train_Y.shape)

torch.Size([9658, 1128])
torch.Size([9658])


#### create the neural network model

1. define the values of the neural network parameters
2. initialize the model
3. define the loss function (in this case, MSE) and optimizer
4. train the model

In [15]:
n_in, n_h, n_out, batch_size = 1128, 2, 1, 1128

model = nn.Sequential(nn.Linear(n_in, n_h),
                     nn.ReLU(),
                     nn.Linear(n_h, n_out),
                     nn.Softmax())

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
optimizer.zero_grad()

for epoch in range(10000):
    # Forward Propagation
    y_pred = model(train_X)
    # Compute and print loss
    loss = criterion(y_pred, train_Y)
#    if(epoch % 100 == 0):
    print('epoch: ', epoch,' loss: ', loss.item())
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()

/usr/local/lib/python3.7/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch:  0  loss:  5.992311954498291
epoch:  1  loss:  5.992311954498291
epoch:  2  loss:  5.992311954498291
epoch:  3  loss:  5.992311954498291
epoch:  4  loss:  5.992311954498291
epoch:  5  loss:  5.992311954498291
epoch:  6  loss:  5.992311954498291
epoch:  7  loss:  5.992311954498291
epoch:  8  loss:  5.992311954498291
epoch:  9  loss:  5.992311954498291


KeyboardInterrupt: 